In [ ]:
!free -h
pd.set_option('display.max_colwidth', None)  # Allow wrapping of long strings


In [19]:
import hytools as ht
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import nbconvert
import time
#from hytools.transform.resampling import apply_resampler
#import brainpipe

In [53]:
pip install spectral


Note: you may need to restart the kernel to use updated packages.


In [69]:
import inspect
print(inspect.getsource(apply_resampler)) 

def apply_resampler(hy_obj,data):
    ''' Apply SCSS correction to a slice of the data

    Args:
        hy_obj (TYPE): DESCRIPTION.
        band (TYPE): DESCRIPTION.
        index (TYPE): DESCRIPTION.

    Returns:
        band (TYPE): DESCRIPTION.

    '''

    interp_types = ['linear', 'nearest', 'nearest-up',
                   'zero', 'slinear', 'quadratic',
                   'cubic']

    #Convert to float
    data = data.astype(np.float32)

    if hy_obj.resampler['type'] == 'gaussian':

        # Load resampling coeffs to memory if needed
        if 'resample_coeffs' not in hy_obj.ancillary.keys():
            in_wave = hy_obj.wavelengths[~hy_obj.bad_bands]
            in_fwhm =hy_obj.fwhm[~hy_obj.bad_bands]
            resample_coeffs = calc_resample_coeffs(in_wave,in_fwhm,
                                             hy_obj.resampler['out_waves'],
                                             hy_obj.resampler['out_fwhm'])
            hy_obj.ancillary['resample_coeffs'] = res

In [58]:
### Loading Earth Lab Spectral Tools

# 1. Enable autoreload in your Jupyter Notebook:

%load_ext autoreload
%autoreload 2

# 2. Import the custom tools module:

import spectral_unmixing_tools_original as el_spectral

# 3. Verify that the tools loaded correctly by printing the module's directory:

print(dir(el_spectral))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['ENVIProcessor', 'GradientBoostingRegressor', '__builtins__', '__cached__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'apply_topo_and_brdf_corrections', 'boosted_quantile_plot', 'boosted_quantile_plot_by_sensor', 'box', 'clean_csv_files_in_subfolders', 'clean_data_and_write_to_csv', 'concatenate_sensors', 'control_function', 'download_neon_file', 'download_neon_flight_lines', 'extract_overlapping_layers_to_2d_dataframe', 'find_raster_files', 'fit_models_with_different_alpha', 'flight_lines_to_envi', 'generate_config_json', 'generate_correction_configs', 'generate_correction_configs_for_directory', 'get_spectral_data_and_wavelengths', 'glob', 'go_forth_and_multiply', 'gpd', 'h5py', 'ht', 'jefe', 'json', 'load_and_combine_rasters', 'load_spectra', 'mask', 'np', 'open_image', 'os', 'pd', 'plot_each_sensor_with_highlight', 'plot_spectral_data', 'plot_with_highlighted_sensors', 'p

In [7]:
import pandas as pd
import os

# Define the base folder and CSV file path
base_folder = "Next_try"
csv_file_path = 'NEON Flightlines to process.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Filter rows where the 'note' column does not contain "provisional"
filtered_df = df[~df['note'].str.contains("provisional", na=False)]

# Group flight lines by unique combinations of site_code, product_code, and year_month
grouped = filtered_df.groupby(['site_code', 'product_code', 'year_month'])

# Create the jefe script
jefe_script = [
    "# el jefe takes 3-5 hours to run and it creates a lot of files.",
    "# You should have 200+GB of RAM and Storage available.",
    f"base_folder = '{base_folder}'\n"
]

for (site_code, product_code, year_month), group in grouped:
    flight_lines = group['flight_line'].tolist()
    jefe_script.append(f"site_code = '{site_code}'")
    jefe_script.append(f"product_code = '{product_code}'")
    jefe_script.append(f"year_month = '{year_month}'")
    jefe_script.append(f"flight_lines = {flight_lines}")
    jefe_script.append("el_spectral.jefe(base_folder, site_code, product_code, year_month, flight_lines)\n")

# Write the script to a Python file
script_file_path = os.path.join(base_folder, 'run_jefe_script.py')
with open(script_file_path, 'w') as f:
    f.write('\n'.join(jefe_script))

print(f"Generated jefe script saved to: {script_file_path}")


Generated jefe script saved to: Next_try/run_jefe_script.py


In [21]:
base_folder = 'non_provisional_flightlines_NIWO'

site_code = 'NIWO'
product_code = 'DP1.30006.001'
year_month = '2020-08'
flight_lines = [
    'NEON_D13_NIWO_DP1_20200731_151902_reflectance'
]

# Run the jefe function with the formatted flight lines
el_spectral.jefe(base_folder, site_code, product_code, year_month, flight_lines)


Processing flight line: NEON_D13_NIWO_DP1_20200731_151902_reflectance
Data retrieved successfully for 2020-08!
Download completed for NEON_D13_NIWO_DP1_20200731_151902_reflectance.h5
Download completed.

Processing: ./NEON_D13_NIWO_DP1_20200731_151902_reflectance.h5
Command executed successfully
Standard Output: Here we GO!

Exporting ancillary data
(HyTools pid=41670) Exported NEON_D13_NIWO_DP1_20200731_151902_reflectance to ENVI format.
(HyTools pid=41670) Ancillary_Imagery/Path_Length /NIWO/Reflectance/Metadata/Ancillary_Imagery/Path_Length
(HyTools pid=41670) to-sensor_Azimuth_Angle /NIWO/Reflectance/Metadata/to-sensor_Azimuth_Angle
(HyTools pid=41670) to-sensor_Zenith_Angle /NIWO/Reflectance/Metadata/to-sensor_Zenith_Angle
(HyTools pid=41670) Logs/Solar_Azimuth_Angle /NIWO/Reflectance/Metadata/Logs/Solar_Azimuth_Angle
(HyTools pid=41670) Logs/Solar_Zenith_Angle /NIWO/Reflectance/Metadata/Logs/Solar_Zenith_Angle
(HyTools pid=41670) Ancillary_Imagery/Slope /NIWO/Reflectance/Metadata

KeyboardInterrupt: 

In [ ]:
base_folder = 'non_provisional_flightlines_RMNP'
site_code = 'RMNP'
product_code = 'DP1.30006.001'
year_month = '2020-07'
flight_lines = [ 'NEON_D10_RMNP_DP1_20200701_164427_reflectance', 
                'NEON_D10_RMNP_DP1_20200701_155155_reflectance']
el_spectral.jefe(base_folder, site_code, product_code, year_month, flight_lines)


Processing flight line: NEON_D10_RMNP_DP1_20200701_164427_reflectance
Data retrieved successfully for 2020-07!
Download completed for NEON_D10_RMNP_DP1_20200701_164427_reflectance.h5
Download completed.

Processing flight line: NEON_D10_RMNP_DP1_20200701_155155_reflectance
Data retrieved successfully for 2020-07!
Download completed for NEON_D10_RMNP_DP1_20200701_155155_reflectance.h5
Download completed.

Processing: ./NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance.h5
Command executed successfully
Standard Output: Here we GO!
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570) G
(HyTools pid=46570) R
(HyTools pid=46570

In [22]:
base_folder = 'non_provisional_flightlines_WREF'
site_code = 'WREF'
product_code = 'DP1.30006.001'
year_month = '2023-06'
flight_lines = [ 
                'NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance']
el_spectral.jefe(base_folder, site_code, product_code, year_month, flight_lines)

Processing flight line: NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance
Data retrieved successfully for 2023-06!
Download completed for NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance.h5
Download completed.

Processing: ./NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance.h5
Command executed successfully
Standard Output: Here we GO!
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G
(HyTools pid=18607) R
(HyTools pid=18607) G


KeyboardInterrupt: 

In [12]:
base_folder = 'non_provisional_flightlines_YELL_2'
site_code = 'YELL'
product_code = 'DP1.30006.001'
year_month = '2022-06'
flight_lines = [ 
                'NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance']
el_spectral.jefe(base_folder, site_code, product_code, year_month, flight_lines)

Processing flight line: NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance
Data retrieved successfully for 2022-06!
Download completed for NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance.h5
Download completed.

Processing: ./NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance.h5
Command executed successfully
Standard Output: Here we GO!
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G
(HyTools pid=11919) R
(HyTools pid=11919) G


Cleaning NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance_spectral_data_all_sensors: 100%|██████████| 87/87 [1:06:40<00:00, 45.98s/it]


Cleaned CSV saved to: non_provisional_flightlines_YELL_2/NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance/NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance_spectral_data_all_sensors_no_NaN.csv

CSV cleaning process completed.

Starting validation of output files in base folder: non_provisional_flightlines_YELL_2



Validating subdirectories: 100%|██████████| 1/1 [00:00<00:00, 12.67it/s]

Subdirectory: NEON_D12_YELL_DP1_L005-1_20220626_directional_reflectance
  All expected files are present and valid.

Jefe finished


In [55]:
import inspect
import spectral
# Get the source code
print(inspect.getsource(spectral.envi.save_image))



def save_image(hdr_file, image, **kwargs):
    '''
    Saves an image to disk.

    Arguments:

        `hdr_file` (str):

            Header file (with ".hdr" extension) name with path.

        `image` (SpyFile object or numpy.ndarray):

            The image to save.

    Keyword Arguments:

        `dtype` (numpy dtype or type string):

            The numpy data type with which to store the image.  For example,
            to store the image in 16-bit unsigned integer format, the argument
            could be any of `numpy.uint16`, "u2", "uint16", or "H".

        `force` (bool):

            If the associated image file or header already exist and `force` is
            True, the files will be overwritten; otherwise, if either of the
            files exist, an exception will be raised.

        `ext` (str or None):

            The extension to use for the image file.  If not specified, the
            default extension ".img" will be used.  If `ext` is an empty
            stri

In [59]:
base_folder = 'non_provisional_flightlines_WREF'
el_spectral.resample_translation_to_other_sensors(base_folder)

Starting tranlation to other sensors
Processing folder: non_provisional_flightlines_WREF/NEON_D16_WREF_DP1_L041-1_20230625_directional_reflectance
Command executed successfully for sensor type: Landsat 5 TM
Standard Output: {'description': 'Resampled hyperspectral data', 'file type': 'ENVI Standard', 'sensor type': 'landsat', 'bands': 6, 'lines': '14396', 'samples': '1098', 'header offset': 0, 'data type': 4, 'interleave': 'bil', 'byte order': 0, 'wavelength units': 'Nanometers', 'wavelength': array([ 485. ,  575. ,  660. ,  837.5, 1697. , 2200. ], dtype=float32), 'map info': ['UTM', '1.000', '1.000', '577927.000', '5082912.000', '1.0000000000e+00', '1.0000000000e+00', '10', 'North', 'WGS-84', 'units=Meters', '0'], 'coordinate system string': 'projcs["utm_zone_10n",geogcs["gcs_wgs_1984",datum["d_wgs_1984",spheroid["wgs_1984",6378137.0,298.257223563]],primem["greenwich",0.0],unit["degree",0.0174532925199433]],projection["transverse_mercator"],parameter["false_easting",500000.0],paramete